# Exercise 11: Topic modeling with latent Dirichlet allocation (LDA) 

In this practical exercise, we will use latent Dirichlet allocation (LDA) to discover the topics prevalent in a document collection, and assign topics to the documents. As discussed in the lecture, an LDA model describes each topic in terms of a distribution over words, and each document as a distribution over topics. The problem setting is unsupervised in the sense that only the text in the documents is observed, and all other variables are latent and need to be inferred by the model.

We will study the <a href="https://archive.ics.uci.edu/ml/datasets/Reuters-21578+Text+Categorization+Collection"> Reuters-21578 </a> document collection, which contains news wire articles that appeared on Reuters in 1987.

First we download and load the Reuters-21578 dataset using the *Natural Language Toolkit* (*NLTK*), which is a platform for handling natural language in Python. It provides access to several text corpora and functions for handling text data. We first download the Reuters-21578 data and import it into the list $articles$.

In [1]:
import nltk
import numpy as np
from nltk.corpus import reuters
nltk.download('reuters')
articles=[]
for doc_id in reuters.fileids():
    articles.append(reuters.raw(doc_id))
articles = articles[:1000]

[nltk_data] Downloading package reuters to /home/madness/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


The variable $articles$ now contains the first 1000 documents in the form of strings.

As discussed in the lecture, LDA is a bag-of-words model, meaning that the probabilistic process for a document does not take into account the order in which the words appear in a document. This means that for performing inference with the LDA model, it suffices to know which words appear in a document and how often each word appears. Therefore we will pass the document collection as a document word frequency matrix. The document word frequency matrix is a 2D array where rows represent documents and columns represent words and each cell counts how many times the specific word appears in a given document.

The following code constructs a document word matrix $tf$, pruning the vocabulary of words such that it only contains words  that:

* have latin characters and are of length 3 or more characters (token_pattern='[a-zA-Z]{3,}'),

* are not english stop words, that is, frequent but uninformative words such as "the" or "a" (stop_words='english')

* occur in at least 0.2% of all documents and at most 95% of all documents (max_df=0.95, min_df=0.002)

and out of these using the 2000 most frequent words (max_features=2000).

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
tf = CountVectorizer( token_pattern='[a-zA-Z]{3,}',max_df=0.95, min_df=0.002,max_features=2000,stop_words='english')
articles_words = tf.fit_transform(articles)
word_index = tf.get_feature_names()

#### Exercise 1:  
Using sklearn.decomposition.LatentDirichletAllocation, train a LDA model with $K=20$ topics on the document collection using the document word frequency matrix *articles_words*. You can find the function documentation <a href="http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html"> here</a>. You can leave the Dirichlet prior parameters $\alpha$ and $\eta$ at their default values of $1/K$. Hint: use the argument enabling multicore processing to increase the speed of inference. Use the argument *random_state=0* to set the random seed of the inference algorithm in order to get reproducible results.

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
k = 20
lda = LatentDirichletAllocation(n_components=k, random_state=0, n_jobs=-1)
lda.fit(articles_words)

/home/madness/venv3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=20, n_jobs=-1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

#### Exercise 2:  
Describe each topic by listing the 10 most probable words in that topic. Please check LatentDirichletAllocation function documentation on how to obtain the word distributions per topic.

In [4]:
n_top_words = 10

# components_[i, j] can be viewed as count that represents the number of times word j was assigned to topic i
# np.argpartition returns the arguments of the k biggest or smallest elements in no order
for topic_idx, topic in enumerate(lda.components_):
        message = "Topic #%d: " % topic_idx
        indices = np.argpartition(topic, -n_top_words)[-n_top_words:]
        indices = indices[np.argsort(-topic[indices])]
        message += " ".join([word_index[i] for i in indices])
        print(message)

Topic #0: tonnes sugar said traders tender export price week sources ecus
Topic #1: mln cts net shr dlrs loss qtr profit revs note
Topic #2: japan trade said japanese exports officials imports washington ministry products
Topic #3: revlon issue macandrews apr bonus gordex jardine matheson cape spencer
Topic #4: oil said gas field crude barrels exxon exploration reserves petroleum
Topic #5: yen half company dividend paid current interim caused motor slow
Topic #6: pct said mln rate billion market bank week days banks
Topic #7: rtz norway unilever chesebrough performance banking wholly subsidiaries buyer bankers
Topic #8: said year pct dlrs mln quarter prices sales company billion
Topic #9: march unemployment workforce government beverage people swedish price faygo bureau
Topic #10: said dlrs dollar billion analysts fed markets deficit trade market
Topic #11: dlrs loans mln non loan brazil income bank quarter trust
Topic #12: bank stg yen mln dollar dealers market bills dollars said
Topi

#### Exercise 3: 
From the inferred topic distributions, we can define the topic distance between two documents $d_1$, $d_2$ to be the Kullback-Leibler divergence between their topic distributions. Let $\theta_{d_j}$ for $j \in \{1,2\}$ be the parameter vector of the categorical topic distributions for documents $d_1$ and $d_2$, and let $\theta_{d_j,i}$ denote their $i$-th component, that is, the probability for topic $i$. 

The Kullback-Leibler divergence is defined by

$$KL(d_1,d_2) = \sum_i{\theta_{d_1,i}}\log{\frac{\theta_{d_1,i}}{\theta_{d_2,i}}}$$

Implement a function *get_similar(doc_id,doc_topic_distribution)* that takes an integer *doc_id* representing the document index and a matrix that gives the distribution over topics for each document. The function should return a list that contains the indices of all documents in the collection ordered by their topic distance to *doc_id*. 


In [5]:
import numpy as np
from tqdm import tqdm

doc_topic_distribution = np.zeros((articles_words.shape[1], 20))
for doc_index in tqdm(range(articles_words.shape[0])):
    doc_topic_distribution[doc_index, :] = lda.transform(articles_words[doc_index, :])

100%|██████████| 1000/1000 [03:10<00:00,  5.24it/s]


In [11]:
from scipy.stats import entropy

def get_similar(doc_id, doc_topic_distribution, top=None):
    doc_scores = np.ones(doc_topic_distribution.shape[0])
    for doc_index, doc in enumerate(doc_topic_distribution):
        doc_scores[doc_index] = entropy(doc_topic_distribution[doc_id], doc)
    # Use argpartition for faster sorting
    # Omit the first element as it is the document itself
    if top is None:
        return np.argsort(doc_scores)[1:]
    else:
        indices = np.argpartition(doc_scores, range(top+1))[1:top+1]
        return indices[np.argsort(doc_scores[indices])]

print(get_similar(0, doc_topic_distribution)[:10])
print(get_similar(0, doc_topic_distribution, top=10))

[728 802 698 521 803 708 187 761 325 202]
[728 802 698 521 803 708 187 761 325 202]


/home/madness/venv3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:2512: RuntimeWarning: invalid value encountered in true_divide
  qk = 1.0*qk / np.sum(qk, axis=0)


#### Exercise 4:  

Get the 10  documents that are most similar to the document with index 1 according to topic distance, and inspect manually whether they are all indeed covering similar content as this document.

To get the distribution of topics over documents, use the LatentDirichletAllocation transform function. It takes the document word matrix and returns an un-normalized document topic distribution, so you have to normalize the matrix before using it.

In [7]:
similar_doc_indices = get_similar(1, doc_topic_distribution, top=10)
for doc_index in similar_doc_indices:
    print(articles[doc_index][:100]) 

BRAZIL SOYBEAN YIELDS SEEN AVERAGE - USDA REPORT
  Based on field travel in the
  Brazilian state of
ROTTERDAM GRAIN HANDLER SAYS PORT BALANCE ROSE
  Graan Elevator Mij, GEM, said its
  balance in port
FAO SEES LOWER GLOBAL WHEAT, GRAIN OUTPUT IN 1987
  The U.N. Food and Agriculture Organisation
  (FA
BRAZIL COTTON CROP LOWER -- USDA REPORT
  Brazil's 1986/87 cotton crop estimate
  has been reduced t
SOVIET UNION TO IMPORT MORE GRAIN IN 86/87-USDA
  The U.S. Agriculture Department
  increased its es
BRAZIL GRAIN HARVEST FACES STORAGE PROBLEMS
  Storage problems with Brazil's record
  grain crop are
DRY SPELL IN PHILIPPINES DAMAGES AGRICULTURE CROPS
  A prolonged dry spell has damaged
  111,350 hec
WEATHER HURTS ITALIAN ORANGES - USDA REPORT
  Unfavorable weather conditions during
  the second wee
INDIA OILSEED OUTPUT FORECAST TO RISE
  India's oilseed output is expected to
  rise to 12.25 mln to
SUBROTO SAYS INDONESIA SUPPORTS TIN PACT EXTENSION
  Mines and Energy Minister Subroto
  co

/home/madness/venv3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:2512: RuntimeWarning: invalid value encountered in true_divide
  qk = 1.0*qk / np.sum(qk, axis=0)
